# Consolidação de Arquivos - Estruturação

### Criação do modelo com todos os atributos possíveis

In [1]:
import duckdb
import pandas as pd

In [2]:
df_base_atributos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [ ]:
df_base_atributos.head(10)

In [4]:
query = """
SELECT
    coluna,
    amostra
FROM (
    SELECT 
            ROW_NUMBER() OVER (PARTITION BY coluna ORDER BY coluna, amostra) AS ID,
            coluna,
            split_part(amostra, ';', 1) amostra
    FROM df_base_atributos
    WHERE coluna IS NOT NULL
    ORDER BY coluna
    ) T1
WHERE ID = 1;
"""

In [5]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    df_modelo_colunas = conexao.sql(query).df()

In [ ]:
df_modelo_colunas

In [7]:
df_modelo_colunas = df_modelo_colunas.pivot(columns="coluna", values="amostra")

In [ ]:
df_modelo_colunas
#df_modelo_colunas.ffill()
#df_modelo_colunas.ffill().bfill().iloc[:1]

In [9]:
df_modelo_colunas = df_modelo_colunas.ffill().bfill().iloc[:1]

In [11]:
(
    df_modelo_colunas.to_json(
        "../resultados/modelos/layout_modelo.json",
        orient="records",
        force_ascii=False
    )
)

### Criando o modelo de dados geral

+ Criando o JsonSchema do modelo

In [12]:
import json


schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "TodasColunas",
    "type": "object",
    "properties": {}
}

with open("../resultados/modelos/layout_modelo.json") as arquivo:
    atributos = json.load(arquivo)
    
for chave in atributos[0].keys():
    schema["properties"][chave] = {"type": "string"}

json_schema = json.dumps(schema, indent=2)

with open("../resultados/modelos/jsonshema_modelo.json", "w") as arquivo:
    json.dump(schema, arquivo, indent=2)

+ Criando a Classe Python do modelo

In [14]:
! mkdir -p ./modelo && \
    datamodel-codegen  \
    --input ../resultados/modelos/jsonshema_modelo.json \
    --input-file-type jsonschema \
    --output ./modelo/modelo.py

### Conversão de todos os arquivos em JSONs válidos.

In [15]:
import os
from pathlib import Path
from secrets import token_hex
from datetime import datetime

import pandas as pd
import duckdb


def gera_arquivo_json(arquivo: str, aba: str, encoding: str ="latin1") -> bool:
    try:
        if arquivo.endswith(".xlsx") or arquivo.endswith(".xls"):
            df = pd.read_excel(arquivo, engine="openpyxl", sheet_name=aba, dtype="str")
        elif arquivo.endswith(".csv") or arquivo.endswith(".txt"):
            df = pd.read_csv(arquivo, encoding=encoding, delimiter=";", dtype="str")
        elif arquivo.endswith(".json"):
            df = pd.read_json(arquivo)
        else:
            raise ValueError("Formato de arquivo não suportado")

        df["caminho_arquivo"] = str(Path(os.path.relpath(arquivo)).parent)
        df["nome_arquivo"] = arquivo.split("/")[-1]
        df["aba_arquivo"] = aba
        df["datahora_processamento"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        df = df.astype("str")

        caminho_arquivo_json = ("../resultados/tmp/jsons")

        if not os.path.exists(caminho_arquivo_json):
            Path(caminho_arquivo_json).mkdir(parents=True, exist_ok=True)

        arquivo_json = f"{arquivo.split('/')[-1].split('.')[0]}-{token_hex(16)}.json"
        df.to_json(f"{caminho_arquivo_json}/{arquivo_json}", orient="records", force_ascii=False)  
        return True

    except Exception as erro:
        raise ValueError(f"Erro na formatação final do arquivo {arquivo}: {erro}")


In [16]:
detalhes_arquivos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [17]:
detalhes_arquivos = detalhes_arquivos.fillna("")

In [ ]:
detalhes_arquivos.head(10)

In [19]:
query = """
SELECT DISTINCT caminho, arquivo, aba
FROM detalhes_arquivos
WHERE status = 'OK';
"""

In [20]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    arquivos_processamento = conexao.sql(query).fetchall()

In [ ]:
arquivos_processamento

In [ ]:
for item in arquivos_processamento:
    arquivo = f"{item[0]}/{item[1]}"
    aba = item[2]
    gera_arquivo_json(arquivo, aba)
    print(f"Arquivo: {arquivo} convertido com Sucesso!")

### Consolidação de todos arquivos JSONs

In [23]:
!export PYTHONPATH=$(realpath $(pwd)/..)

In [24]:
from modelo.modelo import TodasColunas

In [26]:
def lista_arquivos(diretorio: str):
    try:
        lista_arquivos = []

        for root, _dir, arquivos in os.walk(diretorio):
            for arquivo in arquivos:
                if not arquivo.endswith(".gitkeep"):
                    lista_arquivos.append(f"{root}/{arquivo}")

        if len(lista_arquivos) == 0:
            raise TypeError()

        return lista_arquivos

    except Exception as _:
        return False

In [27]:
def consolidacao(arquivos_json: list):
    try:
        consolicacao_tmp = []

        def _carrega_json(arquivo_json: str):
            with open(arquivo_json, "r+", encoding="utf-8") as arquivo:
                return json.load(arquivo)

        for arquivo_json in arquivos_json:
            dados_json = _carrega_json(arquivo_json)

            for registro in dados_json:
                dados_consolidados = TodasColunas(**registro).dict()
                consolicacao_tmp.append(dados_consolidados)

        df_consolicacao_tmp = pd.DataFrame(consolicacao_tmp)
        caminho = "../resultados/consolidacao"
        arquivo = "consolidacao_estruturacao.csv"
        
        if not os.path.exists(caminho):
            Path(caminho).mkdir(parents=True, exist_ok=True)

        df_consolicacao_tmp.to_csv(
            f"{caminho}/{arquivo}",
            sep=";",
            index=False
        )
    except Exception as erro:
        print(f"Erro ao consolidar os arquivos JSONs: {erro}")

In [28]:
arquivos_json = lista_arquivos("../resultados/tmp")

In [ ]:
arquivos_json

In [31]:
consolidacao(arquivos_json)